# Sensitivity test: full Maxey-Riley equation vs slow manifold MR equation
creation date: 19/06/2025  
author: Meike Bos  
Description: In this notebook we compare the trajectories of particles advected with the full Maxey-Riley(MR) equation vs paticles advected with the slow manifold (SM) MR equation  
We do not include the history term (not possible for the SM MR eq). We compare simulations with a constant Rep = 0, 450 and a flexible Rep.   

Comparing will be
- separation distance 




In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

# import needed packages
import sys
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from datetime import datetime, timedelta




sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions_xr import trajectory_length, Haversine_list, calc_tidal_av
from analysis_functions import make_PDF, Haversine, make_lognormal_PDF
from particle_characteristics_functions import factor_drag_white1991, factor_drag_Schiller1933, factor_drag_morrison2013, Re_particle, stokes_relaxation_time

# plotstyle: 
plt.style.use('../python_style_Meike.mplstyle')


Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.9, 5))

In [ ]:
# set needed constants

Rearth = 6371 * 10**3 # in m,
deg2rad = np.pi / 180.
sec_in_hours= 3600
diameter = 0.2 #m
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water


In [ ]:
# files 
Replist = [0,10,100,450,1000] 
coriolis = True
gradient = True
B = 0.68
tau = 2994.76 #2759.97
runtime =  timedelta(days=2)# timedelta(days=10)
land_handling = 'anti_beaching'
nparticles = 88347 # 52511
chunck_time = 100
loc = 'NWES'
displacement = 300 # m 

starttimes = [datetime(2023, 9, 1, 0, 0, 0, 0)]


base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_tracer  = (base_directory + '{particle_type}/{loc}_'
                        'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                        'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')

basefile_tracer_random = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_d{d:04d}_{land_handling}.zarr')

basefiles={'tracer':basefile_tracer,
           'tracer_random':basefile_tracer_random,
           'inertial_Rep_constant':basefile_Rep_constant,
           'inertial_SM_Rep_constant':basefile_Rep_constant, 
           'inertial_SM_drag_Rep':basefile_Rep_drag,
           'inertial_drag_Rep':basefile_Rep_drag}

particle_types = ['tracer','tracer_random','inertial_SM_drag_Rep','inertial_drag_Rep','inertial_SM_Rep_constant','inertial_Rep_constant']#,'inertial_Rep_constant'] # 
names = {'tracer':'tracer',
         'tracer_random':'displaced tracer',
         'inertial_Rep_constant':'MR',
           'inertial_SM_Rep_constant':'SM MR', 
           'inertial_SM_drag_Rep':'SM MR flexible Re$_p$',
           'inertial_drag_Rep':'MR flexible Re$_p$'}


In [ ]:
# read in data

# read in data
data ={}
for pt in particle_types:
    data[pt]={}
    for coriolis in [True]:
        data[pt][coriolis]={}
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
            for Rep in Replist:
                data[pt][coriolis][Rep]={}
        else:
            data[pt][coriolis][None]={}



for pt in particle_types:
    for coriolis in [True]:
        for starttime in starttimes:
            # print(starttime)
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            
            if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
                for Rep in Replist:
                    file = basefiles[pt].format(loc=loc,
                                                y_s=starttime.year,
                                                m_s=starttime.month,
                                                d_s=starttime.day,
                                                y_e=endtime.year,
                                                m_e=endtime.month,
                                                d_e=endtime.day,
                                                B = int(B * 1000), 
                                                tau = int(tau ),
                                                land_handling = land_handling, 
                                                coriolis = coriolis,
                                                gradient = gradient,
                                                particle_type = pt,
                                                Rep = Rep)
                    ds = xr.open_dataset(file,
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)

                    data[pt][coriolis][Rep][date]= ds 
            else:
                # print (pt)
                file = basefiles[pt].format(loc=loc,
                                            y_s=starttime.year,
                                            m_s=starttime.month,
                                            d_s=starttime.day,
                                            y_e=endtime.year,
                                            m_e=endtime.month,
                                            d_e=endtime.day,
                                            land_handling = land_handling, 
                                            coriolis = coriolis,
                                            gradient = gradient,
                                            d=displacement,
                                            B = int(B * 1000), 
                                            tau = int(tau ),
                                            particle_type = pt)
                ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['z'],
                                    decode_times=False) #,decode_cf=False)

                data[pt][coriolis][None][date]= ds 



In [ ]:
# plot selection of trajectories
Blues = colormaps['Blues']

fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.3, 0.95, 5))
colorlist=['k','grey','cornflowerblue','blue']

year=2023
month=9
date = f'{year:04d}/{month:02d}'
legend= []
coriolis = True

ax.coastlines()
idstart =25#25 #600#100#25#6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#98
idstep = 1
idend = idstart+1 * idstep
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
markers=['-','--','-','--','-','--']
for pt, marker in zip(particle_types,markers):
        print(pt)


        



        # ax.legend(particle_types,loc=(1,0.2))
        # pt = 'inertial_SM_Rep_constant'
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
                for Rep, color2 in zip(Replist,Magmalist):
                        ax.plot(data[pt][coriolis][Rep][date].lon[idstart:idend:idstep,0::1].T,
                                data[pt][coriolis][Rep][date].lat[idstart:idend:idstep,0::1].T,
                                marker,
                                color=color2);

                        legend.append(names[pt]+' Re$_p$={:d}'.format(Rep)) #simtype[pt]+
        elif(pt in ('tracer','tracer_random')):
                ax.plot(data[pt][coriolis][None][date].lon[idstart,0::1].T,
                data[pt][coriolis][None][date].lat[idstart,0::1].T,
                marker,
                color='k',zorder=20);
                legend.append(names[pt])
        else:
                ax.plot(data[pt][coriolis][None][date].lon[idstart,0::1].T,
                data[pt][coriolis][None][date].lat[idstart,0::1].T,
                marker,
                color='cornflowerblue',zorder=20);
                legend.append(names[pt])
ax.legend(legend,fontsize=18, handlelength = 1.5,loc=(1,0.))
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
# ax.set_title(simtype[pt])

# ax.set_xlim(-15,10)
# ax.set_ylim(42,62)
# ax.set_xlim(-14.6,-10.5)
# ax.set_ylim(54.4,56.1)
# if(pt == 'inertial_SM_Rep_constant'):
#         fig.tight_layout()
#         fig.savefig('../figures/poster/trajectory_ID25_rep.pdf')
# conclusion Rep = 0, 1, 10 clearly different (but themselves very similar), 100 di

In [ ]:
# add tidal average lons and lats 
window=25
startnan=np.full((nparticles,25),np.nan)
endnan=np.full((nparticles,80),np.nan)
Tend=720
coriolis=True
for starttime in starttimes:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'

    for pt in particle_types:
        print(pt)
        
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
             for Rep in Replist:
                da_lon_tidal_av = calc_tidal_av(data[pt][coriolis][Rep][date].lon,window)
                da_lat_tidal_av = calc_tidal_av(data[pt][coriolis][Rep][date].lat,window)

                data[pt][coriolis][Rep][date] = data[pt][coriolis][Rep][date].assign(lon_tidal_av = da_lon_tidal_av)
                data[pt][coriolis][Rep][date] = data[pt][coriolis][Rep][date].assign(lat_tidal_av = da_lat_tidal_av)
            
        else:
            da_lon_tidal_av = calc_tidal_av(data[pt][coriolis][None][date].lon,window)
            da_lat_tidal_av = calc_tidal_av(data[pt][coriolis][None][date].lat,window)

            data[pt][coriolis][None][date] = data[pt][coriolis][None][date].assign(lon_tidal_av = da_lon_tidal_av)
            data[pt][coriolis][None][date] = data[pt][coriolis][None][date].assign(lat_tidal_av = da_lat_tidal_av)
           

In [ ]:
# along track distance
for pt in particle_types: # loop over all particle types
    for starttime in starttimes: # loop over all starttime
        year = starttime.year
        month = starttime.month
        date = f'{year:04d}/{month:02d}'
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
            for Rep in Replist:
                ds =data[pt][coriolis][Rep][date]
                traj_length = trajectory_length(ds.lon, ds.lat)
                ds = ds.assign(traj_length = traj_length)
                # traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:], ds.lat_tidal_av[:,window:])
                # ds = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
                data[pt][coriolis][Rep][date] = ds
    
        else:
            ds=data[pt][coriolis][None][date]
            traj_length = trajectory_length(ds.lon, ds.lat)
            ds = ds.assign(traj_length = traj_length)
            # traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:],ds.lat_tidal_av[:,window:])
            # ds = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
            data[pt][coriolis][None][date] = ds
            


In [ ]:
#  plot PDF of total trajectory length
fig,ax=plt.subplots()

Tmax = 48

legend=[]
# Tmax=500
for pt, marker in zip(particle_types,markers):




    if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
        for Rep, color2 in zip(Replist,Magmalist[::2]): 
            legend.append(names[pt]+' Re$_p^{\\mathrm{In}}$ ='+' {:d}'.format(Rep))
            data_arrays = list(data[pt][coriolis][Rep].values())
            combined_data = xr.concat(data_arrays, dim="starting_days")

            average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),101, norm=True,vmin=0,vmax=600)
            ax.plot(bins,pdf,marker,color=color2)
    elif(pt in ('tracer','tracer_random')):
        legend.append(names[pt])
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days")

        average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),101, norm=True,vmin=0,vmax=600)


        ax.plot(bins,pdf,marker,color='k',zorder=20)
    else:
        legend.append(names[pt])
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days")

        average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),101, norm=True,vmin=0,vmax=600)


        ax.plot(bins,pdf,marker,color='cornflowerblue',zorder=20)
# ax.set_title(simtype[pt])
ax.legend(legend,fontsize=18)
ax.set_ylabel("PDF")
ax.set_xlabel("trajectory length (km)")


    #tidal_av
   


# for particle_type in particle_types:
#     bins, pdf = make_PDF(arrays[particle_type+'_tidal_av'],401, norm=True)
#     ax.plot(bins,pdf,'--',color=color_particles[particle_type])
#     legend.append(particle_type+' tidal av')



In [ ]:
# plot PDF difference trajecory length and rest of particles
i =0
fig, ax = plt.subplots()
Tmax=48
legend = []
Rep=0
diff = data['tracer'][coriolis][None][date].traj_length-data['tracer_random'][coriolis][None][date].traj_length
bins, pdf = make_PDF( diff[:,Tmax-1].values.flatten(),41, norm=True,vmin = -10,vmax = 10)
ax.plot(bins,pdf,'-',color='grey', zorder = 20)
legend.append('tracer - displaced tracer')


diff = data['inertial_drag_Rep'][coriolis][None][date].traj_length-data['inertial_SM_drag_Rep'][coriolis][None][date].traj_length
bins, pdf = make_PDF( diff[:,Tmax-1].values.flatten(),41, norm=True,vmin = -10,vmax = 10)
ax.plot(bins,pdf,'--',color='cornflowerblue', zorder = i)
legend.append('MR - SM MR, flexible Re$_p$')

for Rep, color2 in zip(Replist, Magmalist):

    diff = data['inertial_Rep_constant'][coriolis][Rep][date].traj_length-data['inertial_SM_Rep_constant'][coriolis][Rep][date].traj_length
    bins, pdf = make_PDF( diff[:,Tmax-1].values.flatten(),41, norm=True,vmin = -10,vmax = 10)
    ax.plot(bins,pdf,'-.',color=color2, zorder = i)
    legend.append('MR - SM MR, Re$_p = $'+f'{Rep:d}')






    # ax.set_title(simtype[pt])
ax.legend(legend,handlelength = 1.5,fontsize=16,loc=2)#,loc=(1,0.2))
ax.set_ylabel("PDF")
ax.set_xlabel("trajecory length difference (km)")
    # ax.set_xlim(-50,50)
# ax.set_yscale('log')

# for Rep = 0, the full MR moves further! 

In [ ]:
# Relative dispersion


for starttime in starttimes:
    print(starttime)
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}' #'inertial_SM_drag_Rep','inertial_Rep_constant','inertial_SM_Rep_constant']

    for pt1, pt2 in zip(particle_types[::2],particle_types[1::2]):
        if(pt1 == 'inertial_SM_Rep_constant'):
            for Rep in Replist:
                ds_Rep_SMMR = data[pt1][coriolis][Rep][date]
                ds_Rep_MR = data[pt2][coriolis][Rep][date]
                dist= Haversine(ds_Rep_MR['lon'],ds_Rep_MR['lat'],
                                    ds_Rep_SMMR['lon'],ds_Rep_SMMR['lat'])
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].assign(dist = dist)
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].rename({'dist':'dist_MR_full'})
                data[pt2][coriolis][Rep][date]=data[pt2][coriolis][Rep][date].assign(dist = dist)
                data[pt2][coriolis][Rep][date]=data[pt2][coriolis][Rep][date].rename({'dist':'dist_MR_full'})

                #tial av
                # dist_tidal_av= Haversine(ds_Rep_MR['lon_tidal_av'],ds_Rep_MR['lat_tidal_av'],
                #                     ds_Rep_SMMR['lon_tidal_av'],ds_Rep_SMMR['lat_tidal_av'])
                # data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].assign(dist_tidal_av = dist_tidal_av)
                # data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].rename({'dist_tidal_av':'dist_MR_full_tidal_av'})
                # data[pt2][coriolis][Rep][date]=data[pt2][coriolis][Rep][date].assign(dist_tidal_av = dist_tidal_av)
                # data[pt2][coriolis][Rep][date]=data[pt2][coriolis][Rep][date].rename({'dist_tidal_av':'dist_MR_full_tidal_av'})
        elif(pt1 == 'tracer'):
            ds = data[pt1][coriolis][None][date]
            ds_random = data[pt2][coriolis][None][date]
            dist= Haversine(ds['lon'],ds['lat'],
                                    ds_random['lon'],ds_random['lat'])
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].assign(dist = dist)
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].rename({'dist':'dist_tracer_random'})
            data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].assign(dist = dist)
            data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].rename({'dist':'dist_tracer_random'})

        else:
            ds_SMMR = data[pt1][coriolis][None][date]
            ds_MR = data[pt2][coriolis][None][date]
            dist= Haversine(ds_MR['lon'],ds_MR['lat'],
                                    ds_SMMR['lon'],ds_SMMR['lat'])
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].assign(dist = dist)
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].rename({'dist':'dist_MR_full'})
            data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].assign(dist = dist)
            data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].rename({'dist':'dist_MR_full'})

            # #tial av
            # dist_tidal_av= Haversine(ds_MR['lon_tidal_av'],ds_MR['lat_tidal_av'],
            #                         ds_SMMR['lon_tidal_av'],ds_SMMR['lat_tidal_av'])
            # data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].assign(dist_tidal_av = dist_tidal_av)
            # data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].rename({'dist_tidal_av':'dist_MR_full_tidal_av'})
            # data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].assign(dist_tidal_av = dist_tidal_av)
            # data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].rename({'dist_tidal_av':'dist_MR_full_tidal_av'})



In [ ]:
# Check particles that shoot of for inertial_drag_Rep
tid = 700
fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ds = data['inertial_Rep_constant'][coriolis][450][date]

# rep_data  = Re_particle(Uslip = np.sqrt(ds.uslip**2+ds.vslip**2), diameter=diameter,kinematic_viscosity=kinematic_viscosity_water)
ax.scatter(ds.lon[:,0],ds.lat[:,0],c=ds['dist_MR_full_tidal_av'][:,tid],s=0.4,vmin=0, vmax=200)

In [ ]:
# data['inertial_Rep_constant'][coriolis][450][date]

In [ ]:
#check that particles do not shoot off anymore
ds = data['inertial_drag_Rep'][coriolis][None][date]
mask = ds['lat'] < 45

# Find which trajectories have any observation with lon > 60
selected_trajectories = mask.any(dim='obs')


# Use the mask to select those trajectories from the orig
ds_selected = ds.sel(trajectory=selected_trajectories)

# no shooting off particles (trajectorys is 0)
print(ds_selected)

In [ ]:
# checking several month flexible Rep

Tmax=48
tlist=np.arange(0,Tmax,1)/24
fig,ax=plt.subplots()
ax2 = ax.twinx()
legend=[]
for pt1 in particle_types[::2]:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    if(pt1 == 'inertial_SM_Rep_constant'):
            for Rep,color2 in zip(Replist,Magmalist):
                ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_MR_full'].mean(dim='trajectory',skipna=True)[0:Tmax],'-.',color=color2)
                legend.append('MR and SM MR, Re$_p$= '+f'{Rep}')
    elif(pt1 == 'tracer'):
        ax2.plot(tlist,data[pt1][coriolis][None][date]['dist_tracer_random'].mean(dim='trajectory',skipna=True)[0:Tmax],'-',color='grey')
        # legend.append('tracer and displaced tracer')
    else:
        ax.plot(tlist,data[pt1][coriolis][None][date]['dist_MR_full'].mean(dim='trajectory',skipna=True)[0:Tmax],'--',color='cornflowerblue')
        legend.append('MR and SM MR, flexible Re$_p$')
ax.legend(legend,fontsize=16,handlelength = 1.5)
ax.set_xlabel('time [days]')
ax.set_ylabel('relative distance [km]')
ax2.set_ylabel('relative distance tracer \n and displaced tracer[km]',color='grey')

ax.set_ylim(-0.05,1.25)
ax2.set_ylim(0.25,1.55)
ax2.tick_params(axis='y', colors='grey')
ax2.spines['right'].set_color('grey')
ax2.set_yticks([0.3,0.5,0.7,0.9,1.1,1.3,1.5],[0.3,0.5,0.7,0.9,1.1,1.3,1.5])

# ax.set_xscale('log')
# ax.set_yscale('log')
# check other definition now I subtracted starting separation distance (300 m) for the tracer to show show that the 
# the dispersive trend is bigger for the randomly displaced tracer, but might be taht the relative dispersion is a better
# measure for this?


In [ ]:
# calculate tau and epsilon

U_flow = 1 #m/s
L_flow = Rearth * (1/36 * deg2rad)  # resolution grid
T_flow = L_flow/ U_flow 
tau_coriolis = 1/(2 * (7.2921159 * 10**-5) * np.sin(50*np.pi/180)) * (2*(1-B))/(1+2*B)
print(tau_coriolis)
for Rep in Replist:
    factor = factor_drag_white1991(Rep)
    tau_stokes = stokes_relaxation_time(0.25,kinematic_viscosity_water,B)
    tau = tau_stokes/factor
    print(f'tau(Rep = {Rep})={tau}')
    epsilon = tau /T_flow# tau_coriolis
    print(epsilon)
    # shortest timescale is 1h because that is the resolution, also resoluiton 1/36 deg 


    

In [ ]:
T_flow

In [ ]:
deg2rad

In [ ]:
L = 